Imports

In [299]:
from pyspark.sql import SparkSession
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql.window import Window #for ranking
from pyspark.sql.functions import lit, mean, stddev_pop
from pyspark.sql.functions import collect_set, collect_list
from pyspark.sql.functions import struct
from pyspark.sql.functions import slice
from pyspark.sql.functions import col
from pyspark.sql.functions import desc
from pyspark.sql.functions import udf
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import DecimalType, ArrayType, IntegerType, FloatType
import pyspark.sql.functions as F
from pyspark.sql.functions import avg, broadcast, when

Define cosine similarity and weighted avg functions

In [300]:
# cosine similarity function
def cosine_similarity_udf(a, b):
    dot_product = sum([x * y for x, y in zip(a, b)])
    norm_a = sum([x**2 for x in a])**0.5
    norm_b = sum([x**2 for x in b])**0.5
    return dot_product / (norm_a * norm_b)


def weighted_avg_features(ratings, combined_vectors):
    if not ratings or not combined_vectors:
        return []

    weighted_sum = [0] * len(combined_vectors[0])
    total_weight = 0

    for rating, combined_vector in zip(ratings, combined_vectors):
        weight = float(rating)
        total_weight += weight
        weighted_sum = [ws + weight * f for ws, f in zip(weighted_sum, combined_vector)]

    if total_weight == 0:
        return weighted_sum

    return [ws / total_weight for ws in weighted_sum]

Load Dataset

In [301]:
spark = SparkSession.builder.appName('ReadMySQL') \
.config("spark.driver.memory", "32g") \
.config("spark.sql.pivotMaxValues", "1000000") \
.config("spark.jars", "C:\\Program Files (x86)\\MySQL\\Connector J 8.0\\mysql-connector-j-8.0.32.jar") \
.getOrCreate()

# sql = "select * from 01_sampled_games_2v2 WHERE playtime_forever IS NOT NULL AND playtime_forever > 0"
sql = sql = """
SELECT p.steamid, p.appid, p.playtime_2weeks, p.playtime_forever, p.dateretrieved, g.genre, d.Developer, pb.Publisher
FROM 01_sampled_games_2v2 AS p
JOIN games_genres AS g ON p.appid = g.appid
JOIN games_developers AS d ON p.appid = d.appid
JOIN games_publishers AS pb ON p.appid = pb.appid
WHERE p.playtime_forever IS NOT NULL AND p.playtime_forever > 0
"""
database = "steam"
user = "root"
password = "root"
server = "127.0.0.1"
port = 3333
jdbc_url = f"jdbc:mysql://{server}:{port}/{database}"
jdbc_driver = "com.mysql.cj.jdbc.Driver"

# Create a data frame by reading data from Oracle via JDBC
df = spark.read.format("jdbc") \
    .option("url", jdbc_url) \
    .option("query", sql) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", jdbc_driver) \
    .load()

df = df.drop("playtime_2weeks", "dateretrieved")

In [302]:
# count number of rows in the dataframe
row_count = df.count()
# print the row count
print("Dataframe has", row_count, " rows.")
df.show()

Dataframe has 212589  rows.
+-----------------+-----+----------------+----------+--------------------+--------------------+
|          steamid|appid|playtime_forever|     genre|           Developer|           Publisher|
+-----------------+-----+----------------+----------+--------------------+--------------------+
|76561197960268000|  300|             109|    Action|               Valve|               Valve|
|76561197960268000| 1300|              94|    Action|Ritual Entertainment|Ritual Entertainment|
|76561197960268000| 2100|             110|    Action|      Arkane Studios|             Ubisoft|
|76561197960268000| 2100|             110|       RPG|      Arkane Studios|             Ubisoft|
|76561197960268000| 4000|             152|     Indie|   Facepunch Studios|               Valve|
|76561197960268000| 4000|             152|Simulation|   Facepunch Studios|               Valve|
|76561197960268000| 2600|              59|    Action|        Troika Games|          Activision|
|76561197960

Build item profiles

In [303]:
# build the item profiles
# Group the data by 'appid' and collect the genres for each game into a list
games_genres_df = df.groupBy("appid").agg(collect_set("genre").alias("genres"))
# Group the data by 'appid' and collect the developers for each game into a list
games_developers_df = df.groupBy("appid").agg(collect_set("Developer").alias("developers"))
# Group the data by 'appid' and collect the publishers for each game into a list
games_publishers_df = df.groupBy("appid").agg(collect_set("Publisher").alias("publishers"))

# Create a list of unique genres
unique_genres = sorted(df.select("genre").distinct().rdd.flatMap(lambda x: x).collect())
# Create a list of unique developers
unique_developers = sorted(df.select("Developer").distinct().rdd.flatMap(lambda x: x).collect())
# Create a list of unique publishers
unique_publishers = sorted(df.select("Publisher").distinct().rdd.flatMap(lambda x: x).collect())


# Define a UDF to create a binary vector for each game's genres
@udf(returnType=ArrayType(IntegerType()))
def genre_vector(genres):
    return [1 if genre in genres else 0 for genre in unique_genres]

# Define a UDF to create a binary vector for each game's developer
@udf(returnType=ArrayType(IntegerType()))
def developer_vector(developers):
    return [1 if developer in developers else 0 for developer in unique_developers]

# Define a UDF to create a binary vector for each game's publisher
@udf(returnType=ArrayType(IntegerType()))
def publisher_vector(publishers):
    return [1 if publisher in publishers else 0 for publisher in unique_publishers]


# Add a new column 'genre_vector' to the DataFrame
# the genre vector will now have a 1 for each genre that the game belongs to
games_genres_df = games_genres_df.withColumn("genre_vector", genre_vector("genres"))
# Add a new column 'developer_vector' to the DataFrame
games_developers_df = games_developers_df.withColumn("developer_vector", developer_vector("developers"))
# Add a new column 'publisher_vector' to the DataFrame
games_publishers_df = games_publishers_df.withColumn("publisher_vector", publisher_vector("publishers"))

# games_genres_df.show(truncate=False)
# Join the main DataFrame with the games_genres_df on appid to include the genre_vector
df = df.join(broadcast(games_genres_df.select("appid", "genre_vector")), on="appid")
# Join the main DataFrame with the games_developers_df
df = df.join(broadcast(games_developers_df.select("appid", "developer_vector")), on="appid")
# Join the main DataFrame with the games_publishers_df
df = df.join(broadcast(games_publishers_df.select("appid", "publisher_vector")), on="appid")

df.show()

+-----+-----------------+----------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|appid|          steamid|playtime_forever|     genre|           Developer|           Publisher|        genre_vector|    developer_vector|    publisher_vector|
+-----+-----------------+----------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  300|76561197960268000|             109|    Action|               Valve|               Valve|[1, 0, 0, 0, 0, 0...|[0, 0, 0, 0, 0, 0...|[0, 0, 0, 0, 0, 0...|
| 1300|76561197960268000|              94|    Action|Ritual Entertainment|Ritual Entertainment|[1, 0, 0, 0, 0, 0...|[0, 0, 0, 0, 0, 0...|[0, 0, 0, 0, 0, 0...|
| 2100|76561197960268000|             110|    Action|      Arkane Studios|             Ubisoft|[1, 0, 0, 0, 0, 0...|[0, 0, 0, 0, 0, 0...|[0, 0, 0, 0, 0, 0...|
| 2100|76561197960268000|             110|    

Normalization

In [304]:
# Calculate the per-game mean and standard deviation of the playtime column
game_stats = df.filter(col("playtime_forever") > 0).groupBy("appid").agg(
    mean("playtime_forever").alias("game_mean_playtime"),
    stddev_pop("playtime_forever").alias("game_stddev_playtime")
)

# Calculate the per-steamid playtime average
user_playtime_avg = df.filter(col("playtime_forever") > 0).groupBy("steamid").agg(avg("playtime_forever")).withColumnRenamed("avg(playtime_forever)", "user_playtime_avg")

# Join the user_playtime_avg dataframe with the main dataframe
df = df.join(user_playtime_avg, "steamid").distinct()

# Join the game_stats dataframe with the main dataframe
df = df.join(game_stats, "appid").distinct()

# Calculate the scaling factor based on the ratio of user playtime to game mean playtime
df = df.withColumn("scaling_factor", col("playtime_forever") / col("user_playtime_avg"))

# Calculate the adjusted cut points
df = df.withColumn("cut_point_1", when(col("game_mean_playtime") - (col("game_stddev_playtime") * 0.5 * col("scaling_factor")) > 0, col("game_mean_playtime") - (col("game_stddev_playtime") * 0.5 * col("scaling_factor"))).otherwise(0))
df = df.withColumn("cut_point_2", col("game_mean_playtime") * col("scaling_factor"))
df = df.withColumn("cut_point_3", col("game_mean_playtime") + (col("game_stddev_playtime") * 0.5 * col("scaling_factor")))
df = df.withColumn("cut_point_4", col("game_mean_playtime") + col("game_stddev_playtime") * col("scaling_factor"))

# Assign ratings based on adjusted cut points
df = df.withColumn(
    "ratings",
    when(col("playtime_forever") <= col("cut_point_1"), lit(1))
    .when((col("playtime_forever") > col("cut_point_1")) & (col("playtime_forever") <= col("cut_point_2")), lit(2))
    .when((col("playtime_forever") > col("cut_point_2")) & (col("playtime_forever") <= col("cut_point_3")), lit(3))
    .when((col("playtime_forever") > col("cut_point_3")) & (col("playtime_forever") <= col("cut_point_4")), lit(4))
    .otherwise(lit(5))
)

# Update the user profile calculation to use the new ratings column
user_aggregated_data = df.groupBy("steamid").agg(
    collect_list("genre_vector").alias("genres_list"),
    collect_list("ratings").alias("ratings_list")
)

# Show df with new changes(without genres)
# Drop the genre_vector and genre columns from the DataFrame
df_without_info = df.drop("genre_vector", "genre", "developer_vector", "Developer", "publisher_vector", "Publisher")

# Show the DataFrame without the genre_vector and genre columns
df_without_info.show()
#df.show()

+------+-----------------+----------------+------------------+------------------+--------------------+--------------------+------------------+------------------+------------------+------------------+-------+
| appid|          steamid|playtime_forever| user_playtime_avg|game_mean_playtime|game_stddev_playtime|      scaling_factor|       cut_point_1|       cut_point_2|       cut_point_3|       cut_point_4|ratings|
+------+-----------------+----------------+------------------+------------------+--------------------+--------------------+------------------+------------------+------------------+------------------+-------+
| 92800|76561197960268000|              14| 278.2970297029703|271.90725806451616|   710.5034416471758|0.050305962715241216|254.03597824223945|13.678556386197021|289.77853788679283|307.64981770906957|      1|
| 49600|76561197960274000|             427|371.29906542056074| 387.4182389937107|   551.3170775921039|  1.1500163608447231|  70.4064093717011| 445.5373133324184| 704.43

Build user profile off this

In [305]:
# combine vectors

# Define a UDF to combine the vectors
@udf(returnType=ArrayType(IntegerType()))
def combined_vector(genre_vector, developer_vector, publisher_vector):
    return genre_vector + developer_vector + publisher_vector

# Add 'combined_vector' to the DataFrame
df = df.withColumn("combined_vector", combined_vector("genre_vector", "developer_vector", "publisher_vector"))

# Update the user profile calculation to use the new ratings column and include developer and publisher vectors
user_aggregated_data = df.groupBy("steamid").agg(
    collect_list("combined_vector").alias("combined_vectors_list"),  
    collect_list("ratings").alias("ratings_list")
)

# Define a UDF to calculate the weighted average of genre vectors
weighted_avg_features_udf = udf(weighted_avg_features, ArrayType(FloatType()))

# Calculate the user profile as the weighted average of rated item profiles (combined genre, developer, and publisher vectors)
user_profiles = user_aggregated_data.withColumn("user_profile", weighted_avg_features_udf("ratings_list", "combined_vectors_list"))

user_profiles.show()

+-----------------+---------------------+--------------------+--------------------+
|          steamid|combined_vectors_list|        ratings_list|        user_profile|
+-----------------+---------------------+--------------------+--------------------+
|76561197960334000| [[1, 0, 0, 0, 0, ...|[1, 2, 1, 2, 2, 1...|[1.0, 0.25, 0.0, ...|
|76561197960338000| [[1, 0, 0, 0, 0, ...|                 [2]|[1.0, 0.0, 0.0, 0...|
|76561197960342000| [[1, 1, 0, 0, 0, ...|[1, 2, 1, 1, 2, 1...|[1.0, 0.3846154, ...|
|76561197960425000| [[0, 0, 0, 0, 0, ...|           [1, 2, 1]|[0.5, 0.0, 0.0, 0...|
|76561197960458000| [[1, 0, 0, 0, 0, ...|[1, 1, 2, 2, 1, 1...|[1.0, 0.0, 0.0, 0...|
|76561197960476000| [[1, 0, 0, 0, 0, ...|              [2, 3]|[1.0, 0.0, 0.0, 0...|
|76561197960884000| [[1, 0, 0, 0, 0, ...|[1, 1, 1, 2, 1, 3...|[1.0, 0.3888889, ...|
|76561197961366000| [[1, 0, 0, 0, 0, ...|[1, 1, 1, 1, 3, 1...|[0.7906977, 0.116...|
|76561197961433000| [[1, 0, 0, 0, 0, ...|        [5, 5, 5, 5]|[1.0, 0.0, 0.0

In [306]:
# Select a steam id and create a list of games that user has already played

In [307]:
# Filter user_profiles for steam_id "76561197960421000"
user_profiles = user_profiles.filter(user_profiles.steamid == "76561197960421000")
# filter for games with playtime
played_games = df.filter(df.steamid == "76561197960421000").select("appid").distinct()

Cosine similarity

In [308]:
# prediction heuristics
# calculate cosine distance of an item and user profile

# 1. create udf for cosine similarity
cosine_similarity = udf(cosine_similarity_udf, FloatType())
# create dataframe with combined vectors
games_combined_vectors_df = df.select("appid", "combined_vector").distinct()
# exclude the games played from games_combined
games_combined_vectors_df = games_combined_vectors_df.join(played_games, "appid", how="left_anti")

# limit the number of rows for testing
user_profiles = user_profiles.limit(1)
# cross join the the combined vectors with the user_profiles
cross_joined = games_combined_vectors_df.crossJoin(user_profiles)

# calculate the cosine similarity between each item and user
recommendations = cross_joined.withColumn(
    "similarity", cosine_similarity("combined_vector", "user_profile")
)

# sort based on similarity score
sorted_recommendations = recommendations.sort(desc("similarity"))

sorted_recommendations.show(10)

+------+--------------------+-----------------+---------------------+--------------------+--------------------+----------+
| appid|     combined_vector|          steamid|combined_vectors_list|        ratings_list|        user_profile|similarity|
+------+--------------------+-----------------+---------------------+--------------------+--------------------+----------+
|  8000|[1, 1, 0, 0, 0, 0...|76561197960421000| [[0, 0, 0, 0, 0, ...|[3, 1, 1, 1, 1, 1...|[0.443038, 0.0632...|0.42611712|
|227600|[0, 1, 0, 1, 0, 0...|76561197960421000| [[0, 0, 0, 0, 0, ...|[3, 1, 1, 1, 1, 1...|[0.443038, 0.0632...| 0.4224751|
|107200|[1, 0, 0, 0, 0, 0...|76561197960421000| [[0, 0, 0, 0, 0, ...|[3, 1, 1, 1, 1, 1...|[0.443038, 0.0632...|0.41154903|
| 65800|[1, 0, 0, 0, 0, 0...|76561197960421000| [[0, 0, 0, 0, 0, ...|[3, 1, 1, 1, 1, 1...|[0.443038, 0.0632...|0.41154903|
| 99300|[1, 0, 0, 0, 0, 0...|76561197960421000| [[0, 0, 0, 0, 0, ...|[3, 1, 1, 1, 1, 1...|[0.443038, 0.0632...|0.38114467|
| 29800|[1, 1, 0

recommender

In [309]:
# Create a window by steamid and similarity to get ranking
window_spec = Window.partitionBy("steamid").orderBy(desc("similarity"))

ranked_recommendations = sorted_recommendations.withColumn("rank", F.row_number().over(window_spec))

top_10_recommendations = ranked_recommendations.filter(ranked_recommendations.rank <= 10)
top_10_recommendations.show(10)

+------+--------------------+-----------------+---------------------+--------------------+--------------------+----------+----+
| appid|     combined_vector|          steamid|combined_vectors_list|        ratings_list|        user_profile|similarity|rank|
+------+--------------------+-----------------+---------------------+--------------------+--------------------+----------+----+
|  8000|[1, 1, 0, 0, 0, 0...|76561197960421000| [[0, 0, 0, 0, 0, ...|[3, 1, 1, 1, 1, 1...|[0.443038, 0.0632...|0.42611712|   1|
|227600|[0, 1, 0, 1, 0, 0...|76561197960421000| [[0, 0, 0, 0, 0, ...|[3, 1, 1, 1, 1, 1...|[0.443038, 0.0632...| 0.4224751|   2|
|107200|[1, 0, 0, 0, 0, 0...|76561197960421000| [[0, 0, 0, 0, 0, ...|[3, 1, 1, 1, 1, 1...|[0.443038, 0.0632...|0.41154903|   3|
| 65800|[1, 0, 0, 0, 0, 0...|76561197960421000| [[0, 0, 0, 0, 0, ...|[3, 1, 1, 1, 1, 1...|[0.443038, 0.0632...|0.41154903|   4|
| 99300|[1, 0, 0, 0, 0, 0...|76561197960421000| [[0, 0, 0, 0, 0, ...|[3, 1, 1, 1, 1, 1...|[0.443038, 0.0